# MODIS snow metrics

MODIS snow metrics are downloaded as multi-band rasters, each band is a
metric and rasters are for years: 2001-2019.

Steps:
1. Create new raster by year filtered on band 10 cells equal to 32.
2. Use zonal statistics to calculate average lcld by watershed.


# 1.1 create dictionary with path and year for all modis rasters

import arcpy
import os

modis_folder = r"W:\GIS\MODIS_Snow_Metrics"
modis_rasters = {}

arcpy.env.workspace = modis_folder
rasters = arcpy.ListRasters()

#remove extra 2011 raster. not sure what it is.
rasters = [x for x in rasters if "_c.tif" not in x]
rasters = [x for x in rasters if "byyear.tif" not in x]
print(rasters)

for raster in rasters:
    # path = os.path.join(modis_folder, raster)
    year = raster[0:4]
    modis_rasters[year] = path

print(len(modis_rasters))
print(modis_rasters)

# 1.2 create new set of rasters after filtering on band 10 value
# for land and continuous snow cover

import arcpy, os
from arcpy.sa import *

arcpy.env.overwriteOutput = True

for key, value in modis_rasters.items():
    print("Creating band_5 mask for: " + key)
    #create mask by converting band_10 to 1/0
    band_10 = Raster(value + "/Band_10")
    mask1 = Con(band_10 == 32, 1)
    #extract band_5 to new raster using new mask
    extract1 = ExtractByMask(value + "/Band_5", mask1)
    extract1.save("W://GIS//MODIS_Snow_Metrics//LCLD_rasters//" + key + "_lcld_32.tif")

# 1.3 create dictionary with path and year for all lcld rasters

import arcpy
import os

lcld_folder = r"W:\GIS\MODIS_Snow_Metrics\LCLD_rasters"
lcld_rasters = {}

arcpy.env.workspace = lcld_folder
rasters = arcpy.ListRasters()

for raster in rasters:
    path = os.path.join(lcld_folder, raster)
    year = raster[0:4]
    lcld_rasters[year] = path

print(len(lcld_rasters))
print(lcld_rasters)

In [ ]:
# 2.1 get average lcld by watershed using zonal statistics as table
# Note there is an error for very small watersheds that don't intersect cell center
# Used try except to skip error, buffer by 150 meters and try again
# MODIS cell size is 500 m so that should work

import arcpy, os, re
from arcpy.sa import *
arcpy.env.overwriteOutput = True

baseFolder = r"W:\GIS\AKSSF"
# regions = ["Kodiak", "Cook_Inlet", "Copper_River", "Bristol_Bay", "Prince_William_Sound"]
regions = ["Copper_River"]

for region in regions:
    print("Starting zonal statistics for region: " + region)
    wtd_dataset = baseFolder + "\\" + region + "\\" + region + ".gdb\\Watersheds"
    arcpy.env.workspace = wtd_dataset
    wtds = arcpy.ListFeatureClasses()
    for wtd in wtds:
        print("starting zonal statistics for wtd: " + wtd)
        wtd_path = os.path.join(wtd_dataset, wtd)
        for key, value in lcld_rasters.items():
            try:
                gdb = baseFolder + "\\" + region + "\\" + region + ".gdb"
                out_table = gdb + "\\" + wtd + "_" + key + "_" + "tbl"
                arcpy.ia.ZonalStatisticsAsTable(wtd_path, "OBJECTID", value, out_table, "DATA", "MEAN", "CURRENT_SLICE", 90, "AUTO_DETECT")
            except:
                buffer = wtd_path + "buf"
                arcpy.Buffer_analysis(wtd_path, buffer, "150 Meters")
                arcpy.ia.ZonalStatisticsAsTable(buffer, "OBJECTID", value, out_table, "DATA", "MEAN", "CURRENT_SLICE", 90, "AUTO_DETECT")

In [ ]:
# 2.2 read in tables, add year and watershed, and merge

import arcpy, os, re
from arcpy.sa import *
arcpy.env.overwriteOutput = True

baseFolder = r"W:\GIS\AKSSF"
regions = ["Kodiak", "Cook_Inlet", "Copper_River", "Bristol_Bay", "Prince_William_Sound"]

for region in regions:
    gdb = baseFolder + "\\" + region + "\\" + region + ".gdb"
    arcpy.env.workspace = gdb
    modis_tables = arcpy.ListTables()
    # print(modis_tables)
    for table in modis_tables:
        in_table = os.path.join(gdb, table)
        arcpy.AddField_management(in_table, "year", "SHORT")
        arcpy.AddField_management(in_table, "cat_ID", "LONG")
        m = re.findall('_(.*)_', table, re.MULTILINE)
        for a in m:
            cat_id = re.split("\_", a)[0]
            year = re.split("\_", a)[1]
        arcpy.CalculateField_management(in_table, "year", year)
        arcpy.CalculateField_management(in_table, "cat_id", cat_id)
    arcpy.Merge_management(modis_tables, "modis_summary_table")


In [ ]:
# 2.3 delete all the modis tables except the final summary table

import arcpy, os

baseFolder = r"W:\GIS\AKSSF"
regions = ["Kodiak", "Cook_Inlet", "Copper_River", "Bristol_Bay", "Prince_William_Sound"]

for region in regions:
    gdb = baseFolder + "\\" + region + "\\" + region + ".gdb"
    arcpy.env.workspace = gdb
    tables = arcpy.ListTables()
    tables = [x for x in tables if "modis_summary_table" not in x]
    tables = [x for x in tables if "vaa_merge" not in x]
    arcpy.Delete_management(tables)